In [46]:
import numpy as np
import pandas as pd

## Format temperature anomalies

In [47]:
def clear_file(file_path, clear_file_path):
    marker = []
    with open(file_path, "r") as file:
        content = file.readlines()
    for i in range(len(content)):
        if content[i][0] == "%":
            marker.append(i)
    with open(clear_file_path, "a") as file:
        for i in range(len(content)): # loop into the list
            if not i in marker: # if the index is not in marker list, then continue writing into file
                file.writelines(content[i])

In [48]:
path_raw_temp_data = 'data/raw/temperature/'
path_processed_temp_data = 'data/processed/temperature/'
path_global_land_ocean_file = 'global-land-ocean.txt'

In [49]:
clear_file(path_raw_temp_data+path_global_land_ocean_file, path_processed_temp_data+path_global_land_ocean_file)
data = pd.read_fwf(path_processed_temp_data+path_global_land_ocean_file)
data.columns = ["Year", "Annual Anomaly", "Annual Unc.", "Five-year Anomaly", "Five-year Unc.", "Annual Anomaly", "Annual Unc.", "Five-year Anomaly", "Five-year Unc."]

In [50]:
data.head(3)

,Year,Annual Anomaly,Annual Unc.,Five-year Anomaly,Five-year Unc.,Annual Anomaly,Annual Unc.,Five-year Anomaly,Five-year Unc.
0,1851,-0.327,0.169,NaN,NaN,-0.324,0.154,NaN,NaN
1,1852,-0.300,0.170,-0.334,0.149,-0.286,0.154,-0.323,0.134
2,1853,-0.306,0.180,-0.302,0.148,-0.294,0.159,-0.292,0.132
